<a href="https://colab.research.google.com/github/Rohan-Bha/Special-Topcis-MIS-IntroToAI2026/blob/main/Lab_3_Part_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai -q
from openai import OpenAI
import re

client = OpenAI(api_key="API_Key")
print('Setup complete.')

Setup complete.


In [2]:
SOURCE_ARTICLE = """
Title: AI Adoption in Healthcare: A 2024 Review of Benefits, Risks, and Equity Concerns

Abstract:
Artificial intelligence is being integrated into healthcare settings at an accelerating pace.
A 2024 review of 42 peer-reviewed studies found that AI diagnostic tools demonstrated
a mean accuracy improvement of 14% over baseline clinician performance in radiology tasks.
However, 67% of studies reported bias in AI systems against underrepresented racial groups,
with false-negative rates up to 23% higher for Black patients in dermatology applications.

Key Findings:
1. AI-powered early detection systems reduced late-stage cancer diagnoses by 19% in pilot hospitals.
2. Administrative AI tools (scheduling, billing) reduced hospital overhead costs by an average of $1.2M per year.
3. Clinician trust in AI recommendations was low (38% adoption rate) unless explainability features were included.
4. Regulatory frameworks lag significantly behind deployment: only 12% of AI tools in use had FDA clearance.
5. Rural hospitals showed the highest potential benefit but the lowest adoption due to infrastructure gaps.

Recommendations:
The authors recommend mandatory bias audits for all clinical AI tools before deployment,
investment in AI literacy training for clinicians, and policy reform to expedite FDA review
processes without sacrificing safety. The paper calls for equity-centered AI development
frameworks to ensure that the benefits of AI do not disproportionately favor already
well-resourced healthcare institutions.
"""

print('Source article loaded.')

Source article loaded.


In [3]:
INITIAL_SYSTEM = """You are a helpful assistant. Summarize articles when asked."""

INITIAL_USER = f"""Summarize the following research article.

{SOURCE_ARTICLE}"""

response_v1 = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[
        {'role': 'system', 'content': INITIAL_SYSTEM},
        {'role': 'user', 'content': INITIAL_USER}
    ],
    temperature=0.5,
    max_tokens=400
)

summary_v1 = response_v1.choices[0].message.content.strip()
print('=== BEFORE: INITIAL SUMMARY (v1) ===')
print(summary_v1)
print(f'\nWord count: {len(summary_v1.split())}')

=== BEFORE: INITIAL SUMMARY (v1) ===
The article titled "AI Adoption in Healthcare: A 2024 Review of Benefits, Risks, and Equity Concerns" examines the integration of artificial intelligence (AI) in healthcare, highlighting both its advantages and challenges. A review of 42 peer-reviewed studies revealed that AI diagnostic tools improved accuracy in radiology by an average of 14% compared to clinicians. However, a significant concern was identified: 67% of studies reported biases in AI systems, particularly affecting underrepresented racial groups, with false-negative rates for Black patients in dermatology being up to 23% higher.

Key findings include:
1. AI early detection systems led to a 19% reduction in late-stage cancer diagnoses.
2. Administrative AI tools cut hospital overhead costs by approximately $1.2 million annually.
3. Clinician trust in AI was low, with only a 38% adoption rate unless AI systems included explainability features.
4. A notable gap was found in regulatory o

In [4]:
SELF_REFLECTION_PROMPT = f"""You wrote the following summary of a research article:

---SUMMARY---
{summary_v1}
---END SUMMARY---

Now critically evaluate your own summary against these REQUIRED criteria:

CRITERIA:
1. LENGTH: Must be between 80-120 words. Score: PASS or FAIL (include actual word count).
2. ACCURACY: Must include at least 3 specific statistics from the article (e.g. percentages, dollar amounts). Score: PASS or FAIL (list which stats are included).
3. AUDIENCE: Must be written for a non-technical healthcare executive — no jargon, no undefined acronyms. Score: PASS or FAIL with examples.
4. STRUCTURE: Must follow this exact format: 1 sentence context → 2-3 sentence findings → 1 sentence recommendation. Score: PASS or FAIL.
5. EQUITY: Must explicitly mention the bias/equity concern from the article. Score: PASS or FAIL.
6. TONE: Must be neutral and objective, not promotional. Score: PASS or FAIL.

Respond with:
CRITIQUE:
- For each criterion: [criterion name]: [PASS/FAIL] — [brief explanation]

OVERALL_SCORE: X/6 criteria passed

IMPROVEMENT_PLAN: List the specific changes needed to pass all criteria.

IMPROVED_SUMMARY: Write the fully revised summary that passes ALL 6 criteria."""

print('Self-reflection prompt ready.')
print(f'Word count of v1 summary: {len(summary_v1.split())}')

Self-reflection prompt ready.
Word count of v1 summary: 218


In [5]:
response_v2 = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[
        {
            'role': 'system',
            'content': 'You are a rigorous editorial assistant. When asked to critique your own writing, be honest and specific. Do not give vague feedback. Always provide an improved version that addresses every failing criterion.'
        },
        {'role': 'user', 'content': INITIAL_USER},
        {'role': 'assistant', 'content': summary_v1},
        {'role': 'user', 'content': SELF_REFLECTION_PROMPT}
    ],
    temperature=0.3,
    max_tokens=900
)

reflection_output = response_v2.choices[0].message.content.strip()
print('=== SELF-REFLECTION OUTPUT ===')
print(reflection_output)

=== SELF-REFLECTION OUTPUT ===
CRITIQUE:
- LENGTH: FAIL — The summary is 189 words, exceeding the maximum limit of 120 words.
- ACCURACY: PASS — The summary includes specific statistics: 14% accuracy improvement, 19% reduction in late-stage cancer diagnoses, and $1.2 million annual cost savings.
- AUDIENCE: PASS — The language is accessible, avoiding jargon and undefined acronyms suitable for a non-technical healthcare executive.
- STRUCTURE: PASS — The summary follows the required format: context, findings, and recommendations.
- EQUITY: PASS — The summary explicitly mentions bias in AI systems affecting underrepresented racial groups and provides specific statistics regarding false-negative rates for Black patients.
- TONE: PASS — The tone is neutral and objective, presenting information without promotional language.

OVERALL_SCORE: 5/6 criteria passed

IMPROVEMENT_PLAN:
1. Reduce the length of the summary to meet the 80-120 word requirement while retaining key information.
2. Ensure

In [6]:
match = re.search(r'IMPROVED_SUMMARY[:\s]+(.*)', reflection_output, re.DOTALL)
if match:
    summary_v2 = match.group(1).strip()
else:
    summary_v2 = '[Could not auto-extract — see full output above]'

print('=' * 60)
print('BEFORE vs AFTER')
print('=' * 60)
print(f'\n📄 BEFORE (v1) [{len(summary_v1.split())} words]:')
print(summary_v1)
print(f'\n✅ AFTER (v2) [{len(summary_v2.split())} words]:')
print(summary_v2)

BEFORE vs AFTER

📄 BEFORE (v1) [218 words]:
The article titled "AI Adoption in Healthcare: A 2024 Review of Benefits, Risks, and Equity Concerns" examines the integration of artificial intelligence (AI) in healthcare, highlighting both its advantages and challenges. A review of 42 peer-reviewed studies revealed that AI diagnostic tools improved accuracy in radiology by an average of 14% compared to clinicians. However, a significant concern was identified: 67% of studies reported biases in AI systems, particularly affecting underrepresented racial groups, with false-negative rates for Black patients in dermatology being up to 23% higher.

Key findings include:
1. AI early detection systems led to a 19% reduction in late-stage cancer diagnoses.
2. Administrative AI tools cut hospital overhead costs by approximately $1.2 million annually.
3. Clinician trust in AI was low, with only a 38% adoption rate unless AI systems included explainability features.
4. A notable gap was found in regul

In [7]:
VERIFICATION_PROMPT = f"""Check whether this summary passes all 6 criteria:

SUMMARY:
{summary_v2}

CRITERIA:
1. LENGTH: Must be 80-120 words
2. ACCURACY: At least 3 specific statistics
3. AUDIENCE: Non-technical, no undefined jargon or acronyms
4. STRUCTURE: 1 sentence context → 2-3 sentence findings → 1 sentence recommendation
5. EQUITY: Explicitly mentions bias/equity concern
6. TONE: Neutral and objective

For each, respond: [criterion]: PASS or FAIL — [one sentence reason]
Then: FINAL SCORE: X/6"""

verify_response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[
        {'role': 'system', 'content': 'You are a strict quality checker. Be objective and precise.'},
        {'role': 'user', 'content': VERIFICATION_PROMPT}
    ],
    temperature=0.1,
    max_tokens=400
)

verification = verify_response.choices[0].message.content.strip()
print('=== FINAL VERIFICATION ===')
print(verification)

=== FINAL VERIFICATION ===
1. LENGTH: PASS — The summary contains 116 words, which is within the required range of 80-120 words.
2. ACCURACY: PASS — The summary includes three specific statistics: 14% improvement in radiology accuracy, 19% reduction in late-stage cancer diagnoses, and $1.2 million annual savings for hospitals.
3. AUDIENCE: PASS — The summary uses clear language without technical jargon or undefined acronyms, making it accessible to a non-technical audience.
4. STRUCTURE: PASS — The summary follows the required structure: it provides context, presents findings in 2-3 sentences, and concludes with a recommendation.
5. EQUITY: PASS — The summary explicitly mentions bias against underrepresented racial groups and highlights the equity concerns related to AI in healthcare.
6. TONE: PASS — The tone is neutral and objective, presenting facts without emotional language or bias.

FINAL SCORE: 6/6
